<a href="https://colab.research.google.com/github/snakecharmergh/ComLing/blob/master/ComLing_hw2_Ngrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Homework (Task 1)** 
Implement a trigram language model, generate some texts and compare it to the bigram language model we wrote above. Which one gives better texts?

Use the code above as a starting point. You don't really need to change much, but it might be difficult to figure out. Read Jurasky carefully to get a better undestanding. And feel free to ask me any questions.

You can use other corpus if you want.

Hints:
you'll need two start-tags in trigram language model,
use bigrams as rows in the matrix and unigrams as columns,
if the text you generated is just randow words - something is wrong


In [0]:
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from string import punctuation
from collections import Counter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
nltk.download('gutenberg')
nltk.corpus.gutenberg.fileids()


[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [0]:
hamlet = nltk.corpus.gutenberg.raw('shakespeare-hamlet.txt')

In [0]:
def normalize(text):
    normalized_text = [word.strip(punctuation) for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

In [0]:
norm_hamlet = normalize(hamlet)

In [0]:
vocab_hamlet = Counter(norm_hamlet)

In [0]:
# to get bigram frequencies.
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [0]:
sentences_hamlet = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(hamlet)]

In [0]:
unigrams_hamlet = Counter()
bigrams_hamlet = Counter()
trigrams_hamlet = Counter()

for sentence in sentences_hamlet:
    unigrams_hamlet.update(sentence)
    bigrams_hamlet.update(ngrammer(sentence))
    trigrams_hamlet.update(ngrammer(sentence, n=3))

In [0]:
# To create a dictionary of word-index and index-word mapping for the matrix
id2word_unigrams = list(unigrams_hamlet)
word2id_unigrams = {word:i for i, word in enumerate(id2word_unigrams)}

id2word_bigrams = list(bigrams_hamlet)
word2id_bigrams = {word:i for i, word in enumerate(id2word_bigrams)}

In [0]:
matrix_hamlet = np.zeros((len(bigrams_hamlet), 
                   len(unigrams_hamlet)))

for ngram in trigrams_hamlet:
    word1, word2, word3 = ngram.split()
    bigram = '{} {}'.format(word1, word2)
    matrix_hamlet[word2id_bigrams[bigram]][word2id_unigrams[word3]] =  (trigrams_hamlet[ngram]/
                                                                                 bigrams_hamlet[bigram])

In [0]:
matrix_hamlet_bigrams = np.zeros((len(unigrams_hamlet), 
                   len(unigrams_hamlet)))

for ngram in bigrams_hamlet:
    word1, word2 = ngram.split()
    matrix_hamlet_bigrams[word2id_unigrams[word1]][word2id_unigrams[word2]] =  (bigrams_hamlet[ngram]/
                                                                     unigrams_hamlet[word1])

**Bigram-generated text**

In [0]:
def generate(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)

In [0]:
print(generate(matrix_hamlet_bigrams, id2word_unigrams, word2id_unigrams).replace('<end>', '\n'))

i must render vp the morning know him from a sterrill promontory this but who i am still hast doth burne out of gain-giuing as snow qu 
 neuer beleeue so will fight with this i meane my lord i haue told thee hence pursue me you doe your father in doubt that would harrow vp my sword rebellious hell 
 then i will scarce hold it cannot ham 
 not thinke tis true a winde is my lord i thinke there is that lend a lash that i essentially am i haue thoughts ham 
 ham 
 ham 



**Trigram-generated text**

In [0]:
def start_step(text, matrix_2, word2id_bigrams, id2word, start_idx):
    second_word = np.random.choice(matrix_2.shape[1], p=matrix_2[start_idx])
    word = id2word[second_word]
    text.append(word)
    current_bigram = "<start> {}".format(word)
    current_idx = word2id_bigrams[current_bigram]
    return current_bigram, current_idx

def generate(matrix, matrix_2, id2word, word2id_bigrams, word2id_unigrams, n=100):
    text = []
    start_idx = word2id_unigrams["<start>"]
    current_bigram, current_idx = start_step(text, matrix_2, word2id_bigrams, id2word, start_idx)
    for i in range(n):

        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        word = id2word[chosen]
        
        if word == '<end>':
            current_bigram, current_idx = start_step(text, matrix_2, word2id_bigrams, id2word, start_idx)
        else:
            text.append(word)
            current_bigram = "{} {}".format(current_bigram.split()[1], word)
            current_idx = word2id_bigrams[current_bigram]
    
    return ' '.join(text)

In [0]:
print(generate(matrix_hamlet, matrix_hamlet_bigrams, id2word_unigrams, word2id_bigrams, word2id_unigrams, n=100).replace('<end>', '\n'))

exceedingly my lord ham i did verie well my lord begin murderer but orderly to end where i am dead thou liu'st report me and that would be scann'd a villaine dwelling in all i shall liue and feede vpon your maiestie mother you haue heard the cocke crew hor and hath shipped me intill the land and fertile let a beast be lord of beasts and his sandal shoone qu for murther though it make the matter ham for hecuba come let me see another ham euery foole can tell that it is too long but here my father king


Both bigram and trigram language models generated quite meaningful texts. However, trigram model gives better results because the trigram model looks two words into the past, while the bigram model looks only one word into the past. Thus, the trigram model generated more coherent and comprehensible text. The trigram model somehow generated longer meaningful sentencses like: "let me see let me be" or "a speech of fire that faine would blaze" or "she should locke her selfe in ease".


**Homework (Task 2)**

Implement a simple version of Byte-pair-encoding (see first seminar) using gensim.models.Phrases.

Apply gensim.models.Phrases to character sequences instead of word sequences (sentences). Train at least 3 Phrases sequentially. As a result you should get whole words or long character ngrams.


In [0]:
import re, collections
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from nltk import word_tokenize
from gensim.models.phrases import Phrases, Phraser
from gensim.models.word2vec import Text8Corpus
from gensim.test.utils import datapath

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
text = '''Amid the cries of war, death, and poverty, loud sounds deep down of me are rising me up, calling me to find my way out of the rubble, free my own self with the weapon of knowledge and revive my dying country. Armed with hope, I have decided to start my own journey in search for ways to better my life and others’. Bridging the widening gap between Syria and the rest of the world is vital for shaping a better future. Since English speaking countries have a leading role in today’s world and English is the lingua franca, an MA in English Studies is perfect for not only mastering English language but also for getting well-informed about the history, language and culture of such countries.  What can be better than the heart of Europe to be my stop for getting closely in touch with the cultures of those countries?! I found my haven in your prestigious university with its unique MA program to be my final destination for many reasons. First, it is highly acclaimed locally and internationally. Second, the course gives the ability to specialize in linguistics which is interrelated to translation, my PgD. specialization. Furthermore, your academic staff boast international reputation for their professionalism. Moreover, the course is not only fine-tuned to the job market needs but also serves as a springboard into further academic study. It will equip me with up-to-date knowledge, critical perspective, interpretive and analytical skills, argumentative writing, and academic research tools. 
My undergraduate study of English language and literature enabled me to get more acquainted with both English language and culture. I got a more specialized look into translation, which is a means of knowledge transfer and intercultural communication, through my postgraduate diploma in Translation and Arabization. I am a certified freelance translator. I worked as a translator in the Syrian Parliament and as a translation project manager for Syrian Society for Scientific Research. My hard work paid off when I got the chance to work as a translator in a publishing house in Lebanon and as an interpreter in two conferences under the auspices of the League of Arab States. My attention was not only cast on translation-related jobs, but also on life-changing jobs that will help me in achieving my goals of spreading knowledge and enhancing international relationships. As the IAESTE National Secretary and the Head of Student Exchange in the Syrian Ministry of Higher Education, I could help lots of students to pursue their higher studies and get introduced to different cultures. I am proud I had the chance to participate in strengthening intercultural and educational relations as well as in rebuilding hope and transforming lives of Syrians from being helpless refugees into productive active players in the reconstruction of Syria.
Pursuing this program would help realize my goals. First, specializing in linguistics enables me to pursue a career as a translator in an international high profile organization to translate the pains and hopes of the oppressed. I will use translation to get introduced to different fields of knowledge, diverse cultures and the latest developments. This contributes to the reconstruction of Syria. Furthermore, English Studies is vital for finding out points of convergence between Arabs and the rest of the world, correcting misconceptions, and defying stereotypes. It would serve as a springboard for openness and acceptance of the “Other”. It will hopefully contribute to turning the world into a more harmonious, well-informed and secure place to live in. 
In short, my proven abilities would make me a perfect fit. My ambition to further develop myself and my thirst for employing the things I learn in helping others especially Syrians in times of war and later on in rebuilding Syria is the strongest motivation to keep me working relentlessly. This makes me meet the standards of your prestigious institution.'''

In [0]:
sentences = sent_tokenize(text)

In [0]:
sentences

['Amid the cries of war, death, and poverty, loud sounds deep down of me are rising me up, calling me to find my way out of the rubble, free my own self with the weapon of knowledge and revive my dying country.',
 'Armed with hope, I have decided to start my own journey in search for ways to better my life and others’.',
 'Bridging the widening gap between Syria and the rest of the world is vital for shaping a better future.',
 'Since English speaking countries have a leading role in today’s world and English is the lingua franca, an MA in English Studies is perfect for not only mastering English language but also for getting well-informed about the history, language and culture of such countries.',
 'What can be better than the heart of Europe to be my stop for getting closely in touch with the cultures of those countries?!',
 'I found my haven in your prestigious university with its unique MA program to be my final destination for many reasons.',
 'First, it is highly acclaimed local

In [0]:
def split(word): 
    return [char for char in word]

In [0]:
vocab = []
for i in range(len(sentences)):
    vocab.append(split(sentences[i]))

In [0]:
print(vocab)

[['A', 'm', 'i', 'd', ' ', 't', 'h', 'e', ' ', 'c', 'r', 'i', 'e', 's', ' ', 'o', 'f', ' ', 'w', 'a', 'r', ',', ' ', 'd', 'e', 'a', 't', 'h', ',', ' ', 'a', 'n', 'd', ' ', 'p', 'o', 'v', 'e', 'r', 't', 'y', ',', ' ', 'l', 'o', 'u', 'd', ' ', 's', 'o', 'u', 'n', 'd', 's', ' ', 'd', 'e', 'e', 'p', ' ', 'd', 'o', 'w', 'n', ' ', 'o', 'f', ' ', 'm', 'e', ' ', 'a', 'r', 'e', ' ', 'r', 'i', 's', 'i', 'n', 'g', ' ', 'm', 'e', ' ', 'u', 'p', ',', ' ', 'c', 'a', 'l', 'l', 'i', 'n', 'g', ' ', 'm', 'e', ' ', 't', 'o', ' ', 'f', 'i', 'n', 'd', ' ', 'm', 'y', ' ', 'w', 'a', 'y', ' ', 'o', 'u', 't', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 'r', 'u', 'b', 'b', 'l', 'e', ',', ' ', 'f', 'r', 'e', 'e', ' ', 'm', 'y', ' ', 'o', 'w', 'n', ' ', 's', 'e', 'l', 'f', ' ', 'w', 'i', 't', 'h', ' ', 't', 'h', 'e', ' ', 'w', 'e', 'a', 'p', 'o', 'n', ' ', 'o', 'f', ' ', 'k', 'n', 'o', 'w', 'l', 'e', 'd', 'g', 'e', ' ', 'a', 'n', 'd', ' ', 'r', 'e', 'v', 'i', 'v', 'e', ' ', 'm', 'y', ' ', 'd', 'y', 'i', 'n', 'g', ' '

In [0]:
ph = gensim.models.Phrases(vocab, min_count=1, threshold=-1, scoring='npmi')

In [0]:
p = gensim.models.Phrases(ph[vocab], min_count=1, threshold=-1, scoring='npmi')

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
p2= gensim.models.Phrases(p[phrases[vocab]], min_count=1, threshold=0, scoring='npmi')

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
p3=Phrases (p2[p[phrases[vocab]]], min_count=1, threshold=0.1, scoring='npmi')

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
print(list(p3[p2[p[vocab]]]))

[['A_m', 'i_d', ' _t', 'h_e', ' _c', 'r_i', 'e_s', ' _o', 'f_ ', 'w_a', 'r', ',', ' _d', 'e_a', 't_h', ',', ' _a', 'n_d', ' _p', 'o', 'v_e', 'r_t', 'y', ',', ' _l', 'o_u', 'd_ ', 's', 'o_u', 'n_d', 's_ ', 'd_e', 'e_p', ' _d', 'o_w', 'n_ ', 'o_f', ' _m', 'e_ ', 'a_r', 'e_ ', 'r_i', 's', 'i_n', 'g', ' _m', 'e_ ', 'u_p', ',', ' _c', 'a_l', 'l_i', 'n_g', ' _m', 'e_ ', 't_o', ' _f', 'i_n', 'd_ ', 'm', 'y_ ', 'w_a', 'y_ ', 'o_u', 't_ ', 'o_f', ' _t', 'h_e', ' _r', 'u_b', 'b_l', 'e', ',', ' _f', 'r_e', 'e_ ', 'm', 'y_ ', 'o_w', 'n_ ', 's', 'e_l', 'f_ ', 'w_i', 't_h', ' _t', 'h_e', ' _w', 'e_a', 'p_o', 'n_ ', 'o_f', ' _k', 'n_o_w_l', 'e_d', 'g_e', ' _a', 'n_d', ' _r', 'e_v', 'i', 'v_e', ' _m', 'y_ ', 'd_y', 'i_n', 'g', ' _c', 'o_u', 'n_t', 'r_y', '.'], ['A_r', 'm_e', 'd_ ', 'w_i', 't_h', ' _h', 'o_p', 'e', ',', ' ', 'I_ ', 'h_a', 'v_e', ' _d', 'e_c', 'i_d', 'e_d', ' _t', 'o_ ', 's_t', 'a_r', 't_ ', 'm', 'y_ ', 'o_w', 'n_ ', 'j_o', 'u_r', 'n_e', 'y_ ', 'i_n', ' _s', 'e_a_r', 'c_h', ' _f', 'o_r'

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
phrases_input = phrases[vocab]
for i in range(100):
    phrases = gensim.models.Phrases(phrases_input, min_count=1, threshold=-1, scoring='npmi')
    phrases_input = phrases[phrases_input]

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
print(list(phrases[phrases_input]))

[['A_m_i_d_ _t_h_e_ _c_r_i_e_s_ _o_f_ _w_a_r_,_ _d_e_a_t_h_,_ _a_n_d_ _p_o_v_e_r_t_y_,_ _l_o_u_d_ _s_o_u_n_d_s_ _d_e_e_p_ _d_o_w_n_ _o_f_ _m_e_ _a_r_e_ _r_i_s_i_n_g_ _m_e_ _u_p_,_ _c_a_l_l_i_n_g_ _m_e_ _t_o_ _f_i_n_d_ _m_y_ _w_a_y_ _o_u_t_ _o_f_ _t_h_e_ _r_u_b_b_l_e_,_ _f_r_e_e_ _m_y_ _o_w_n_ _s_e_l_f_ _w_i_t_h_ _t_h_e_ _w_e_a_p_o_n_ _o_f_ _k_n_o_w_l_e_d_g_e_ _a_n_d_ _r_e_v_i_v_e_ _m_y_ _d_y_i_n_g_ _c_o_u_n_t_r_y_.'], ['A_r_m_e_d_ _w_i_t_h_ _h_o_p_e_,_ _I_ _h_a_v_e_ _d_e_c_i_d_e_d_ _t_o_ _s_t_a_r_t_ _m_y_ _o_w_n_ _j_o_u_r_n_e_y_ _i_n_ _s_e_a_r_c_h_ _f_o_r_ _w_a_y_s_ _t_o_ _b_e_t_t_e_r_ _m_y_ _l_i_f_e_ _a_n_d_ _o_t_h_e_r_s_’_.'], ['B_r_i_d_g_i_n_g_ _t_h_e_ _w_i_d_e_n_i_n_g_ _g_a_p_ _b_e_t_w_e_e_n_ _S_y_r_i_a_ _a_n_d_ _t_h_e_ _r_e_s_t_ _o_f_ _t_h_e_ _w_o_r_l_d_ _i_s_ _v_i_t_a_l_ _f_o_r_ _s_h_a_p_i_n_g_ _a_ _b_e_t_t_e_r_ _f_u_t_u_r_e_.'], ['S_i_n_c_e_ _E_n_g_l_i_s_h_ _s_p_e_a_k_i_n_g_ _c_o_u_n_t_r_i_e_s_ _h_a_v_e_ _a_ _l_e_a_d_i_n_g_ _r_o_l_e_ _i_n_ _t_o_d_a_y_’_s_ _w_o_r_l_d_ _a_n_d_ _E_

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
